In [20]:
import sys
sys.path.append("C:/Users/javie/Desktop/CVC")

import os
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import pickle
import torch
import trimesh
import numpy as np
from torch import nn
from tqdm.notebook import tnrange, tqdm

from utils.slices import *
from utils.orientaciones import *


In [13]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        pkl_path = "../data/oriented_dataset/mesh_info2.pkl"
        if os.path.exists(pkl_path):
            with open(pkl_path, 'rb') as archivo:
                pkl_object = pickle.load(archivo)
            self.classes = pkl_object[0]
            self.mesh_info = torch.tensor(pkl_object[1])
            self.labels = torch.tensor(pkl_object[2]) #index of self.classes

        else:
            self.classes = os.listdir(root_dir)
            self.mesh_info = []
            self.labels = []
            
            for class_index, class_path in enumerate(self.classes):
                mesh_paths = os.listdir(root_dir + class_path)
                for path in mesh_paths:
                    mesh = trimesh.load(root_dir + class_path + '/' + path)
                    
                    xsec_0 = xsection(mesh, origin_plane=[0,0,0])
                    xsec_1_pos = xsection(mesh, origin_plane=[1,0,0])
                    xsec_1_neg = xsection(mesh, origin_plane=[-1,0,0])

                    xsec_0 = np.column_stack(flip_and_roll(xsec_0[:, 1],xsec_0[:, 2]))
                    xsec_1_pos = np.column_stack(flip_and_roll(xsec_1_pos[:, 1],xsec_1_pos[:, 2]))
                    xsec_1_neg = np.column_stack(flip_and_roll(xsec_1_neg[:, 1],xsec_1_neg[:, 2]))
                    
                    xsec_0 = xsec_0[np.linspace(0,xsec_0.shape[0],21,dtype=np.int16)[:-1]]
                    xsec_1_pos = xsec_1_pos[np.linspace(0,xsec_1_pos.shape[0],21,dtype=np.int16)[:-1]]
                    xsec_1_neg = xsec_1_neg[np.linspace(0,xsec_1_neg.shape[0],21,dtype=np.int16)[:-1]]

                    xsec_0 = xsec_0.flatten()
                    xsec_1_pos = xsec_1_pos.flatten()
                    xsec_1_neg = xsec_1_neg.flatten()
                    
                    descriptors_list = [mesh.area, mesh.volume, mesh.convex_hull.volume, mesh.volume/mesh.convex_hull.volume]
                    descriptors_list.extend(xsec_0)
                    descriptors_list.extend(xsec_1_pos)
                    descriptors_list.extend(xsec_1_neg)
                    self.mesh_info.append(descriptors_list)
                    self.labels.append(class_index)
            
            with open(pkl_path, 'wb') as archivo:
                pickle.dump([self.classes, self.mesh_info, self.labels], archivo)
                
            #self.classes = torch.tensor(self.classes)
            self.mesh_info = torch.tensor(self.mesh_info)
            self.labels = torch.tensor(self.labels)
            
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return self.mesh_info[index], self.labels[index]

In [14]:
dataset = CustomImageDataset("../data/oriented_dataset/Barley STL files/Dundee/")

train_size = int(0.8 * len(dataset))  # 80% de datos de entrenamiento
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

C:\Users\javie\AppData\Local\Temp\ipykernel_20832\3138988362.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.mesh_info = torch.tensor(pkl_object[1])
C:\Users\javie\AppData\Local\Temp\ipykernel_20832\3138988362.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(pkl_object[2]) #index of self.classes


In [15]:
for X, y in train_loader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")

Shape of X [N, C, H, W]: torch.Size([16, 124])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 124])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 124])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 124])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 124])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 124])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 124])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 124])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 124])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 124])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.Size([16, 124])
Shape of y: torch.Size([16]) torch.int64
Shape of X [N, C, H, W]: torch.S

In [5]:
'''# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
'''
# Define model
class GrainClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(124, 512, dtype=torch.double),
            nn.ReLU(),
            nn.Linear(512, 512, dtype=torch.double),
            nn.ReLU(),
            nn.Linear(512, 512, dtype=torch.double),
            nn.ReLU(),
            nn.Linear(512, 512, dtype=torch.double),
            nn.ReLU(),
        )
        self.classification_layer = nn.Linear(512, 7, dtype=torch.double)
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        x = self.classification_layer(x)
        return x

'''model = GrainClassifier().to(device)
print(model)'''

model = GrainClassifier()
print(model)

GrainClassifier(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=124, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): ReLU()
  )
  (classification_layer): Linear(in_features=512, out_features=7, bias=True)
)


In [6]:
'''
    X = torch.rand(10, 124, device=device)
    X *= -1
    logits = model(X)
    pred_probab = nn.Softmax(dim=1)(logits)
    y_pred = pred_probab.argmax(1)
    print(f"Input: {X}")
    print(f"Prob class: {pred_probab}")
    print(f"Predicted class: {y_pred}")'''

'X = torch.rand(10, 124, device=device)\nX *= -1\nlogits = model(X)\npred_probab = nn.Softmax(dim=1)(logits)\ny_pred = pred_probab.argmax(1)\nprint(f"Input: {X}")\nprint(f"Prob class: {pred_probab}")\nprint(f"Predicted class: {y_pred}")'

In [7]:
# Optimizers specified in the torch.optim package
loss_fn = torch.nn.CrossEntropyLoss()
learning_rate = 1e-3
momentum = 0.9
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

In [8]:
def train_one_epoch(dataloader):
    running_loss = 0.
    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
    #print("Total loss: ", running_loss)

    return running_loss

In [9]:
def test_model(dataloader):
    total_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(dataloader):
        inputs, labels = data
        outputs = model(inputs)

        loss = loss_fn(outputs, labels)
        total_loss += loss.item()

        _, predictions = torch.max(outputs.data, 1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)  # Número total de ejemplos

    # Calcula la precisión como el número de predicciones correctas dividido por el total
    accuracy = correct / total
        
    return total_loss, accuracy

In [10]:
max_accuracy = 0
for i in range(5000):
    train_loss = train_one_epoch(train_loader)
    test_loss, accuracy = test_model(test_loader)
    if max_accuracy < accuracy:
        max_accuracy = accuracy
    if i % 25 == 0:
        print("Epoch: " + "%-3i" % i + ", train loss: " + str(train_loss) + ", test loss: " + str(test_loss), ", accuracy: " + str(accuracy))
print(max_accuracy)

Epoch: 0  , train loss: 33.13236400123571, test loss: 8.644609027301673 , accuracy: 0.2608695652173913
Epoch: 25 , train loss: 29.249603168125695, test loss: 8.209316002115175 , accuracy: 0.2608695652173913
Epoch: 50 , train loss: 28.209683565548275, test loss: 8.533362828311159 , accuracy: 0.2028985507246377
Epoch: 75 , train loss: 27.58757585006918, test loss: 8.5207140787995 , accuracy: 0.21739130434782608
Epoch: 100, train loss: 27.49423286165683, test loss: 8.837730228069466 , accuracy: 0.21739130434782608
Epoch: 125, train loss: 27.1392648690767, test loss: 8.558119150565378 , accuracy: 0.2318840579710145
Epoch: 150, train loss: 26.902616886355396, test loss: 8.49139460376711 , accuracy: 0.2318840579710145
Epoch: 175, train loss: 26.189493173109966, test loss: 8.80580346272503 , accuracy: 0.2028985507246377
Epoch: 200, train loss: 27.132250019494045, test loss: 9.037732679339989 , accuracy: 0.2318840579710145
Epoch: 225, train loss: 26.839565155963975, test loss: 8.58851435677458

In [11]:
'''
for inputs, labels in train_loader:
    preds = model(inputs)
    print("outputs: ", torch.argmax(preds, dim=1))
    print("labels: ", labels)
    '''

'\nfor inputs, labels in train_loader:\n    preds = model(inputs)\n    print("outputs: ", torch.argmax(preds, dim=1))\n    print("labels: ", labels)\n    '

In [12]:
print(max_accuracy)

0.5217391304347826
